In [38]:
# Logistic Regression을 python, sklearn, tensorflow로 각각 구현
import numpy as np
import tensorflow as tf
from sklearn import linear_model

# 수치 미분함수
from my_library.machine_learning_library import numerical_derivative

# Raw Data Loading + Data Preprocessing

# Training Data Set
x_data = np.arange(2,21,2).reshape(-1,1)
t_data = np.array([0,0,0,0,0,0,1,1,1,1]).reshape(-1,1)

# 1. python 구현

# Weight & bias
W = np.random.rand(1,1)
b = np.random.rand(1)

# Loss Function
def loss_func(input_obj):
    # input_obj : W와 b를 같이 포함하고 있는 ndarray => [W1,W2,W3, ... , b]
    num_of_bias = b.shape[0]
    input_W = input_obj[:-1 * num_of_bias].reshape(-1,num_of_bias)  # 행렬연산을 하기 위한 W 추출
    input_b = input_obj[-1 * num_of_bias]
    
    # 우리 모델의 예측값 : Linear Regression model(Wx + b) ==> sigmoid 적용
    z = np.dot(x_data,input_W) + input_b
    y = 1 / (1 + np.exp(-1*z))
    
    delta = 1e-7 #  0에 가까운 작은 값을 줌으로써 프로그램의 로그 연산시 무한대로 발산하는 것을 방지
    
    # Cross Entropy
    return -np.sum(t_data * np.log(y+delta) + ((1-t_data) * np.log(1-y+delta)))
    
# Learning rate
learning_rate = 1e-4

# 학습
for step in range(30000):
    
    input_param = np.concatenate((W.ravel(), b.ravel()), axis = 0)
    derivative_result = learning_rate * numerical_derivative(loss_func, input_param)
    
    
    num_of_bias = b.shape[0]
    
    W = W - derivative_result[:-1 * num_of_bias].reshape(-1,num_of_bias)
    b = b - derivative_result[-1 * num_of_bias]
    
    
# Prediction => W,b를 구해서 Logistic Regression Model을 완성
def logistic_predict(x):
    
    z = np.dot(x,W) + b
    y = 1 / (1 + np.exp(-1 * z))
    
    if y < 0.5 :
        result = 0
    else :
        result = 1
    
    return result, y

input_value = np.array([[13]])
result = logistic_predict(input_value)
print('python 결과값')
print('예측결과 : {}, 확률 : {}'.format(result[0],result[1][0,0]))

python 결과값
예측결과 : 1, 확률 : 0.5798057103407672


In [37]:
# 2. sklearn 구현

# Logistic Regression Model 생성
model = linear_model.LogisticRegression()

# Traning Data Set을 이용한 학습
model.fit(x_data,t_data.ravel())

# Prediction
input_value = np.array([[13]])
pred_val = model.predict(input_value)
pred_proba = model.predict_proba(input_value)
print('sklearn 결과값')
print('예측결과 : {}, 확률 : {}'.format(pred_val,pred_proba))

python 결과값
예측결과 : [0], 확률 : [[0.50009391 0.49990609]]


In [40]:
# 3. tensorflow 구현

# Placeholder 생성
X = tf.placeholder(dtype=tf.float32)  # 독립변수가 1개인 경우(simple), shape 명시 x
T = tf.placeholder(dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([1,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='weight')

# Hypothesis
logit = W * X + b
H = tf.sigmoid(logit)

# Loss Function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# Train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    sess.run(train, feed_dict={X : x_data, T : t_data})
    
# Prediction
input_value = np.array([13])
result = sess.run(H, feed_dict={X : input_value})
print('tensorflow 결과값')
print(result)

tensorflow 결과값
[[0.58202696]]
